In [1]:
#Imports
from pyspark.sql.functions import col, udf, unix_timestamp, year, month, dayofmonth
from pyspark.sql.types import DateType, StringType
from pyspark.sql.functions import lpad
from pyspark.sql.functions import concat, lit 
from pyspark.sql.functions import regexp_replace, col
import pyspark.sql.functions as f

#url for account and Key
spark.conf.set("fs.azure.account.key.coeinventorydatalake.dfs.core.windows.net", "sWdPowaV0oY+RUjJuDsou8hDe8n6NX9EzJ/c5y3w4cQj20GnlGLNMeQhjzMxtQcuE9B5wKPhtipsoFYK1C0cJg==")

#Accessing file from datalake in azure storage 
dbutils.fs.ls('abfss://comm@coeinventorydatalake.dfs.core.windows.net/')

Out[1]: [FileInfo(path='abfss://comm@coeinventorydatalake.dfs.core.windows.net/comm-report/', name='comm-report/', size=0),
 FileInfo(path='abfss://comm@coeinventorydatalake.dfs.core.windows.net/comm-report-cleaned/', name='comm-report-cleaned/', size=0)]

### Authorization and Configuration Setup

In [3]:
configs = {"fs.azure.account.auth.type": "OAuth",
           "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
           "fs.azure.account.oauth2.client.id": "50f1bc0d-0ae6-42f2-a416-ae2305402b3d",
           "fs.azure.account.oauth2.client.secret": "2Oht2QJz?7Gd.rYmag-deJClHyfDEj]2",
           "fs.azure.account.oauth2.client.endpoint": "https://login.microsoftonline.com/4e9dbbfb-394a-4583-8810-53f81f819e3b/oauth2/token"}

In [4]:
dbutils.fs.ls('/mnt/comm-report/')

Out[92]: [FileInfo(path='dbfs:/mnt/comm-report/comm-report/', name='comm-report/', size=0),
 FileInfo(path='dbfs:/mnt/comm-report/comm-report-cleaned/', name='comm-report-cleaned/', size=0)]

### Reading in the data from blob storage so we can work with it

In [6]:
df = spark.read.format('csv').options(header = 'true', inferSchema = 'true', delimiter = ',').load('/mnt/comm/comm-report/')

In [7]:
jdbcHostname = "mtwf1.database.windows.net"
jdbcDatabase = "syk_gc"
jdbcPort = 1433

#jdbcUsername = dbutils.secrets.get(scope ='key-vault-secrets',  key = 'sql-username')
#jdbcPassword = dbutils.secrets.get(scope ='key-vault-secrets',  key = 'sql-password')

jdbcUrl = 'jdbc:sqlserver://{0}:{1};database={2}'.format(jdbcHostname, jdbcPort, jdbcDatabase)
connectionProperties = {
  'user' : "gc_admin2",
  'password' : "HQSPe@2+H6n",
  'driver' : 'com.microsoft.sqlserver.jdbc.SQLServerDriver'
}

In [8]:
#bringing in cork cogs data
pushDown_query = """(select* from GIM_Item) as s"""

In [9]:
gimdf = spark.read.jdbc(url=jdbcUrl, table=pushDown_query, properties=connectionProperties)

for c in gimdf.columns:
  gimdf = gimdf.withColumnRenamed(c, c.replace(' ','').lower())
gimdf = gimdf.select('catalognumber', 'normalizedcatalognumber')

In [10]:
#df.select('ITEM_NUMBER').where(col('ITEM_NUMBER').like("0%")).show()

In [11]:
#ISSUE
df = df.select("*", f.regexp_replace(f.col("ITEM_NUMBER"), "[-,./]", "").alias("replaced")).drop('ITEM_NUMBER')

In [12]:
df = df.select('*', f.regexp_replace('replaced', r'^*', '').alias('ITEM_NUMBER'))

In [13]:
#df.select('ITEM_NUMBER').where(col('ITEM_NUMBER').like("0%")).show()

In [14]:
newdf = df.join(gimdf, df.ITEM_NUMBER == gimdf.normalizedcatalognumber, how='inner' )

In [15]:
df = newdf.drop('ITEM_NUMBER')
df = df.withColumnRenamed('catalognumber', 'Item_number')

In [16]:
from pyspark.sql.functions import coalesce
df = df.withColumn('MANUFACTURING_DATE', coalesce('MANUFACTURING_DATE', 'PO_RECEIPT_DATE'))
df = df.withColumn('MANUFACTURING_DATE', coalesce('MANUFACTURING_DATE', 'INTRANSIT_RECEIPT_DATE'))

### Selecting what we need to work with from the database

In [18]:
df2 = df.select('Item_number', 'SERIAL_NUMBER', 'LOT_NUMBER', 'INTRANSIT_RECEIPT_DATE', 'LOT_EXPIRATION_DATE', 'MANUFACTURING_DATE', 'INTRANSIT_RECEIPT_DATE', 'PO_RECEIPT_DATE', 'CUSTOMER_NUMBER', 'ORGANIZATION_ID') 

### adding in lot_serial column and filling it with lot_number

In [20]:
df3 = df2.withColumn('LOT_SERIAL', df2.LOT_NUMBER)

### Filling in the lot_serial column with serial_number when there is not lot_number to place so if there is an na

In [22]:
df4 =df3.withColumn('LOT_SERIAL', coalesce('LOT_SERIAL', 'SERIAL_NUMBER'))

### dropping lot_number and serial_number columns

In [24]:
df4 = df4.drop('LOT_NUMBER', 'SERIAL_NUMBER')

### selecting our dataframe again

In [26]:
df4 = df4.select('Item_number', 'LOT_SERIAL', 'INTRANSIT_RECEIPT_DATE', 'PO_RECEIPT_DATE', 'MANUFACTURING_DATE', 'LOT_EXPIRATION_DATE', 'CUSTOMER_NUMBER', 'ORGANIZATION_ID')

### replacing the dateof columns that were strings as new columns that are the in Date format

In [28]:
df5= df4.withColumn('DateOfManufacture', df4['MANUFACTURING_DATE'].cast(DateType())).drop('MANUFACTURING_DATE')
df6= df5.withColumn('DateOfLotExpiration', df5['LOT_EXPIRATION_DATE'].cast(DateType())).drop('LOT_EXPIRATION_DATE')

###  What this does in creates 3 new fields for each value of the date in the manufacting and expiration date columns and puts the specificed part of the date in the 6 columns we created

In [30]:
df7 = (df6.select(year('DateOfManufacture').alias('ManufacturerYear'),month('DateOfManufacture').alias('ManufactureMonth'),dayofmonth('DateOfManufacture').alias('ManufactureDay'), 'Item_number',  'LOT_SERIAL', 'INTRANSIT_RECEIPT_DATE', 'PO_RECEIPT_DATE','DateOfLotExpiration', 'CUSTOMER_NUMBER', 'ORGANIZATION_ID'))
           

df7 =(df7.select(year('DateOfLotExpiration').alias('ExpirationYear'),month('DateOfLotExpiration').alias('ExpirationMonth'),dayofmonth('DateOfLotExpiration').alias('ExpirationDay'), 'Item_number', 'LOT_SERIAL', 'INTRANSIT_RECEIPT_DATE', 'PO_RECEIPT_DATE', 'ManufacturerYear', 'ManufactureMonth', 'ManufactureDay', 'CUSTOMER_NUMBER', 'ORGANIZATION_ID'))

###  This places in the 0s in our to fill in the day and month colums so 1 reads 01.

In [32]:
df7a = df7.select('Item_number', 'LOT_SERIAL', 'INTRANSIT_RECEIPT_DATE', 'PO_RECEIPT_DATE', 'ManufacturerYear', 'ExpirationYear', 'CUSTOMER_NUMBER', 'ORGANIZATION_ID', lpad(df7.ExpirationMonth, 2, '0').alias('ExpirationMonth'), (lpad(df7.ExpirationDay, 2, '0').alias('ExpirationDay')),(lpad(df7.ManufactureMonth, 2, '0').alias('ManufactureMonth')), (lpad(df7.ManufactureDay, 2, '0').alias('ManufactureDay')))

### concats the 6 fields into two columns that read the manufacture and expiration date

In [34]:
df8 = (df7a.select('*')
      .withColumn('Lot_expiration', concat(col('ExpirationDay'), lit('/'), col('ExpirationMonth'), lit('/'), col('ExpirationYear')))
      .withColumn('Date_of_manufacture', concat(col('ManufactureDay'), lit('/'), col('ManufactureMonth'), lit('/'), col('ManufacturerYear')))
      .drop('ManufacturerYear', 'ManufactureMonth','ManufactureDay', 'ExpirationDay', 'ExpirationMonth', 'ExpirationYear')
      )

### makes nas become empty strings

In [36]:
df9 = df8.na.fill({'Lot_expiration': ''},{'Date_of_manufacture': ''})
df10 = df9.na.fill({'Date_of_manufacture': ''})
df10 = df10.na.fill({'INTRANSIT_RECEIPT_DATE': ''})

### selects entire dataframe

In [38]:
df11 = df10.select('Item_number', 'LOT_SERIAL', 'Lot_expiration', 'INTRANSIT_RECEIPT_DATE', 'PO_RECEIPT_DATE', 'Date_of_manufacture', 'CUSTOMER_NUMBER', 'ORGANIZATION_ID')

###  makes organization_ID a string instead of integer and changes name to ORG_ID

In [40]:
df11 = df11.withColumn('ORG_ID',df11['ORGANIZATION_ID'].cast(StringType())).drop('ORGANIZATION_ID')

### adds in thre columns as strings

In [42]:
df12 = (df11.withColumn('M', lit('1')).withColumn('O', lit('2')).withColumn('R', lit('3')))

### selects dataframe

In [44]:
df13 = df12.select('Item_number', 'LOT_SERIAL', 'Lot_expiration', 'INTRANSIT_RECEIPT_DATE', 'Date_of_manufacture', 'M', 'ORG_ID')

In [45]:
display(df13)

Item_number,LOT_SERIAL,Lot_expiration,INTRANSIT_RECEIPT_DATE,Date_of_manufacture,M,O,CUSTOMER_NUMBER,R,ORG_ID
0233032104,000224004,,,21/05/2020,1,2,132279,3,90
0233032104,000224004,,,21/05/2020,1,2,132279,3,90
0233032104,000224004,,,21/05/2020,1,2,132279,3,90
0233032104,000224004,,,21/05/2020,1,2,132279,3,90
0233032104,000224004,,,21/05/2020,1,2,132279,3,90
0233032104,000224004,,,21/05/2020,1,2,132279,3,90
0233050065,142304,,,22/07/2020,1,2,132279,3,90
0233050065,142304,,,22/07/2020,1,2,132279,3,90
0233050065,142304,,,22/07/2020,1,2,132279,3,90
0233050087,20G5019,,,08/07/2020,1,2,132279,3,90


###  connecting back to our database

In [47]:
jdbcHostname = "mtwf1.database.windows.net"
jdbcDatabase = "mtwf1"
jdbcPort = 1433

 

jdbcUsername = dbutils.secrets.get(scope ='key-vault-secrets',  key = 'sql-username')
jdbcPassword = dbutils.secrets.get(scope ='key-vault-secrets',  key = 'sql-password')

 

jdbcUrl = 'jdbc:sqlserver://{0}:{1};database={2}'.format(jdbcHostname, jdbcPort, jdbcDatabase)
connectionProperties = {
  'user' : jdbcUsername,
  'password' : jdbcPassword,
  'driver' : 'com.microsoft.sqlserver.jdbc.SQLServerDriver'
}

###  writing this as an sql table to our database

In [49]:
df13.write.jdbc(url= jdbcUrl, table= 'comm_org_report', mode= 'append', properties= connectionProperties)